## 简介
本项目提供 事件抽取任务 baseline，比赛详情可查看[比赛详情](https://aistudio.baidu.com/aistudio/competition/detail/32?isFromCcf=true)  
具体流程可以参考 event-extract-baseline-seq-labeling/README.md  
详细代码 可查看baseline目录
本项目提供数据为demo数据，如需使用完整数据可在比赛页面下载。  
**特别提示**：CPU环境运行可能存在内存溢出的问题，建议使用GPU环境运行代码。

In [1]:
# 切换到代码目录
%cd baseline/

/home/aistudio/baseline


In [2]:
# 下载预训练的ERNIE模型
%cd ./model
!wget https://ernie.bj.bcebos.com/ERNIE_1.0_max-len-512.tar.gz --no-check-certificate
!mkdir ERNIE_1.0_max-len-512  # ernie模型保存目录
!tar -zxf ERNIE_1.0_max-len-512.tar.gz -C ERNIE_1.0_max-len-512 
!rm ERNIE_1.0_max-len-512.tar.gz
%cd ../

[Errno 2] No such file or directory: './model'
/home/aistudio/baseline
--2020-09-19 16:21:31--  https://ernie.bj.bcebos.com/ERNIE_1.0_max-len-512.tar.gz
Resolving ernie.bj.bcebos.com (ernie.bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c00:6c21:10ad:0:ff:b00e:67d
Connecting to ernie.bj.bcebos.com (ernie.bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 367271417 (350M) [application/x-gzip]
Saving to: ‘ERNIE_1.0_max-len-512.tar.gz’

ERNIE_1.0_max-len-5 100%[===================>] 350.26M  76.3MB/s    in 5.0s    

2020-09-19 16:21:36 (69.7 MB/s) - ‘ERNIE_1.0_max-len-512.tar.gz’ saved [367271417/367271417]

/home/aistudio


In [3]:
# 处理样本数据
# 将原始格式的数据处理成模型读入所需要的格式，并随机划分训练集（80%）和测试集（20%），在 `./data/` 目录下创建 `train.json`、`dev.json`、`test.json` 文件
!python bin/data_process.py origin_events_process ./data/eet_events.json ./data/

python: can't open file 'bin/data_process.py': [Errno 2] No such file or directory


In [19]:
# 处理schema生成序列标注标签文档
# 触发词识别模型标签，保存到文件 `./dict/vocab_trigger_label_map.txt`
!python bin/data_process.py schema_event_type_process ./dict/event_schema.json ./dict/vocab_trigger_label_map.txt

include event type 4,  create label 9


In [20]:
# 论元角色识别模型标签，保存到文件 `./dict/vocab_roles_label_map.txt`
!python bin/data_process.py schema_role_process ./dict/event_schema.json ./dict/vocab_roles_label_map.txt

include roles 7，create label 15


In [3]:
# 训练触发词识别模型
!sh train_trigger.sh

+ dirname script/train_event_trigger.sh
+ readlink -f script
+ HERE=/home/aistudio/baseline/bin/script
+ cd /home/aistudio/baseline/bin/script/..
+ export CUDA_VISIBLE_DEVICES=0
+ export FLAGS_eager_delete_tensor_gb=0
+ export FLAGS_fraction_of_gpu_memory_to_use=0.3
+ DATA_DIR=/home/aistudio/baseline/bin/script/../../data
+ SAVE_CKPT=/home/aistudio/baseline/bin/script/../../save_model/trigger
+ MODEL_PATH=/home/aistudio/baseline/bin/script/../../model/ERNIE_1.0_max-len-512
+ DICT=/home/aistudio/baseline/bin/script/../../dict
+ python run_event_trigger.py --use_cuda true --do_train true --do_val true --do_test true --batch_size 32 --init_pretraining_params /home/aistudio/baseline/bin/script/../../model/ERNIE_1.0_max-len-512/params --trigger_pred_save_path /home/aistudio/baseline/bin/script/../../save_model/trigger/pred_trigger.json --chunk_scheme IOB --label_map_config /home/aistudio/baseline/bin/script/../../dict/vocab_trigger_label_map.txt --train_set /home/aistudio/baseline/bin/scrip

In [4]:
# 预测触发词结果
!sh predict_trigger.sh

+ dirname script/predict_event_trigger.sh
+ readlink -f script
+ HERE=/home/aistudio/baseline/bin/script
+ cd /home/aistudio/baseline/bin/script/..
+ export CUDA_VISIBLE_DEVICES=0
+ export FLAGS_eager_delete_tensor_gb=0
+ export FLAGS_fraction_of_gpu_memory_to_use=0.3
+ DATA_DIR=/home/aistudio/baseline/bin/script/../../data
+ MODEL_PATH=/home/aistudio/baseline/bin/script/../../model/ERNIE_1.0_max-len-512
+ CKPT_PATH=/home/aistudio/baseline/bin/script/../../save_model/trigger/final_model
+ DICT=/home/aistudio/baseline/bin/script/../../dict
+ python run_event_trigger.py --use_cuda true --do_train false --do_val true --do_test true --batch_size 16 --init_pretraining_params /home/aistudio/baseline/bin/script/../../model/ERNIE_1.0_max-len-512/params --trigger_pred_save_path /home/aistudio/baseline/bin/script/../../save_model/trigger/final_model/../pred_trigger.json --chunk_scheme IOB --label_map_config /home/aistudio/baseline/bin/script/../../dict/vocab_trigger_label_map.txt --train_set /ho

In [5]:
# 训练论元角色识别模型
!sh train_role.sh

+ dirname script/train_event_role.sh
+ readlink -f script
+ HERE=/home/aistudio/baseline/bin/script
+ cd /home/aistudio/baseline/bin/script/..
+ export CUDA_VISIBLE_DEVICES=0
+ export FLAGS_eager_delete_tensor_gb=0
+ export FLAGS_fraction_of_gpu_memory_to_use=0.3
+ DATA_DIR=/home/aistudio/baseline/bin/script/../../data
+ SAVE_CKPT=/home/aistudio/baseline/bin/script/../../save_model/role
+ MODEL_PATH=/home/aistudio/baseline/bin/script/../../model/ERNIE_1.0_max-len-512
+ DICT=/home/aistudio/baseline/bin/script/../../dict
+ python run_event_role.py --use_cuda true --do_train true --do_val true --do_test true --batch_size 32 --init_pretraining_params /home/aistudio/baseline/bin/script/../../model/ERNIE_1.0_max-len-512/params --trigger_pred_save_path /home/aistudio/baseline/bin/script/../../save_model/role/pred_role.json --chunk_scheme IOB --label_map_config /home/aistudio/baseline/bin/script/../../dict/vocab_roles_label_map.txt --train_set /home/aistudio/baseline/bin/script/../../data/trai

In [6]:
# 预测论元角色结果
!sh predict_role.sh

+ dirname script/predict_event_role.sh
+ readlink -f script
+ HERE=/home/aistudio/baseline/bin/script
+ cd /home/aistudio/baseline/bin/script/..
+ export CUDA_VISIBLE_DEVICES=0
+ export FLAGS_eager_delete_tensor_gb=0
+ export FLAGS_fraction_of_gpu_memory_to_use=0.3
+ DATA_DIR=/home/aistudio/baseline/bin/script/../../data
+ MODEL_PATH=/home/aistudio/baseline/bin/script/../../model/ERNIE_1.0_max-len-512
+ CKPT_PATH=/home/aistudio/baseline/bin/script/../../save_model/role/final_model
+ DICT=/home/aistudio/baseline/bin/script/../../dict
+ python run_event_role.py --use_cuda true --do_train false --do_val true --do_test true --batch_size 16 --init_pretraining_params /home/aistudio/baseline/bin/script/../../model/ERNIE_1.0_max-len-512/params --trigger_pred_save_path /home/aistudio/baseline/bin/script/../../save_model/role/final_model/../pred_role.json --chunk_scheme IOB --label_map_config /home/aistudio/baseline/bin/script/../../dict/vocab_roles_label_map.txt --train_set /home/aistudio/basel

In [4]:
# 评估
# 将测试集（`./data/test.json`）转化为评估格式 `./bin/evaluate/test.json`
!python bin/predict_eval_process.py test_data_2_eval ./data/test.json ./bin/evaluate/test.json
# 将预测结果整合并转为评估格式 
!python bin/predict_eval_process.py predict_data_2_eval ./save_model/trigger/pred_trigger.json ./save_model/role/pred_role.json ./dict/event_schema.json ./bin/evaluate/pred.json

test data 2 eval data, inputs 303 outputs 299
predict data 2 eval data start
load trigger predict datas 300 from ./save_model/trigger/pred_trigger.json
load role predict datas 300 from ./save_model/role/pred_role.json
load schema datas 4 from ./dict/event_schema.json
predict data 2 eval data is finished, outputs 279


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 